# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [77]:
import os
import time
import yaml
import re
import json
import requests
import pickle

from openai import OpenAI
import pandas as pd
import numpy as np
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv

# ENV, Config 파일 읽기

In [92]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [93]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# gpt prompt 설정
sentiment_instructions = config['gptapi']['sentiment']['instructions']
sentiment_few_shots = config['gptapi']['sentiment']['few_shot_examples']
theme_instructions = config['gptapi']['theme']['instructions']
theme_few_shots = config['gptapi']['theme']['few_shot_examples']


# 인스턴스 생성

In [19]:
# OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 함수 정의

In [38]:
def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    try:
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages = messages,
            # max_tokens = 150,
            temperature = 0.5,
            stream = False)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# JSON 응답에서 raw_text 항목을 추출하는 함수
def extract_raw_text(content):
    return content.get('raw_text', '')

# JSON 응답에서 issue 항목을 추출하는 함수
def extract_issues(content):
    return content.get('issue', [])

# JSON 응답에서 각 issue에 대한 sentiment 값을 추출하는 함수
def extract_sentiments(content):
    return content.get('sentiment', {})

# JSON 응답에서 sentiment_all 값을 추출하는 함수
def extract_sentiment_all(content):
    return content.get('sentiment_all', '')

# 문자열을 딕셔너리로 변환하는 함수
def convert_to_dict(content):
    try:
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    
# JSON 데이터를 변환하고 추출된 데이터를 DataFrame으로 변환하는 함수
def convert_to_dataframe(content_list):
    data = []
    
    for content in content_list:
        if not content:
            continue  # content가 None이거나 비어 있을 때 건너뜀
        
        content_dict = convert_to_dict(content)
        if not content_dict:
            continue  # JSON 변환에 실패한 경우 건너뜀
        
        raw_text = extract_raw_text(content_dict)
        issues = extract_issues(content_dict)
        sentiments = extract_sentiments(content_dict)
        sentiment_all = extract_sentiment_all(content_dict)
        
        # 각 issue와 sentiment를 별도로 저장
        for issue, sentiment in sentiments.items():
            data.append({
                "raw_text": raw_text,
                "issue": issue,
                "sentiment": sentiment,
                "sentiment_all": sentiment_all
            })
    
    # 데이터가 있을 경우에만 DataFrame 생성
    if data:
        return pd.DataFrame(data)
    else:
        print("No valid data to create DataFrame.")
        return pd.DataFrame()  # 빈 DataFrame 반환
    
def map_issues_to_themes(issues_list, clustering_response):
    themes = []
    for issues in issues_list:
        issue_themes = set()  # 중복을 방지하기 위해 set 사용
        for issue in issues:
            for theme, issue_group in clustering_response.items():
                if issue in issue_group:
                    issue_themes.add(theme)
        themes.append(", ".join(issue_themes))  # 여러 테마가 있을 경우 ","로 결합
    return themes

# 후처리 함수 정의(리스트, 딕셔너리 형태를 문자열로 변환)
def clean_format(value):
    if isinstance(value, list):
        return ', '.join(value)  # 리스트의 경우 ,로 구분하여 문자열로 변환
    if isinstance(value, dict):
        return ', '.join([f"{k}: {v}" for k, v in value.items()])  # 딕셔너리의 경우 key: value 형식으로 변환
    return value  # 이미 문자열인 경우는 그대로 반환

# 파일 읽기

# test 영역

In [8]:
# 처리할 데이터
df = pd.read_csv('data/제공/제공_네이버쇼핑리뷰.csv')
df.head()

,내용,구매처,평점,작성날짜,사용자,상품
0,조아용 조아용 조아용 ㅎㅎ,오아시스486,5,24.05.15.,rorn****,스탠바이미
1,너무 만족하며 사용중입니다,Ader,5,24.05.15.,arde****,스탠바이미
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,Ader,5,24.05.15.,jeon****,스탠바이미
3,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,Ader,5,24.05.15.,pjw9****,스탠바이미
4,살까말까 했는데 좋아요,11번가,4,24.05.15.,m0******,스탠바이미


## test

In [11]:
test = df['내용'][:20]

In [24]:
print(test[8])

주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다


In [26]:
input_raw_text = test[8]
system_message = {"role": "system", "content": sentiment_instructions}
user_message = {"role": "user", "content": f"{sentiment_few_shots}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
messages = [system_message, user_message]
content = get_response(messages)

In [27]:
print(content)

{
  "raw_text": "주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다",
  "issue": [
    "빠른 설치 안내",
    "저렴한 가격"
  ],
  "sentiment": {
    "빠른 설치 안내": "긍정",
    "저렴한 가격": "긍정"
  },
  "sentiment_all": "긍정"
}


In [30]:
content

'{\n  "raw_text": "주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다",\n  "issue": [\n    "빠른 설치 안내",\n    "저렴한 가격"\n  ],\n  "sentiment": {\n    "빠른 설치 안내": "긍정",\n    "저렴한 가격": "긍정"\n  },\n  "sentiment_all": "긍정"\n}'

In [32]:
content_dict = convert_to_dict(content)

In [33]:
raw_text = extract_raw_text(content_dict)
issues = extract_issues(content_dict)
sentiments = extract_sentiments(content_dict)
sentiment_all = extract_sentiment_all(content_dict)

In [34]:
raw_text

'주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다'

In [39]:
result = convert_to_dataframe([content])

In [40]:
result

,tid,raw_text,issue,sentiment,sentiment_all
0,FB-0001,주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다,빠른 설치 안내,긍정,긍정
1,FB-0001,주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다,저렴한 가격,긍정,긍정


In [42]:
test = df['내용'][:20]
print(test)

0                                        조아용 조아용 조아용 ㅎㅎ
1                                        너무 만족하며 사용중입니다
2         4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요
3                          여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요
4                                          살까말까 했는데 좋아요
5                    빠른 배송 감사합니다. LG에서 직잡 배송/설치 했습니다. .
6     자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확...
7     핸드폰으로 ott를 보다보니 항상 아쉬웠는데 고민고민하다질렀어요~^^ 주문후 다음날...
8      주문하고 바로 설치안내문자오더니 토요일에 기사님 오셔서 설치해주셨어요 싸게 잘 샀습니다
9                                너무 예뻐요 배송 잘 해주셔서 고맙습니다
10                                     충전이 번거롭긴 하지만 대만족
11    친절하게 설치해주시구 무었보다 빠르게 설치해주셔서 넘 좋았어요 >__< 좋은가격에 ...
12                                     이동하기 너무 편해서 좋네요.
13      배송 이틀만에 왔구요 와서 조립까지 다 해주고 가셨어요!저렴한 가격에 적극 추천합니다
14          너무 좋아요무겁지만 바퀴가 있어 이동이 용이 하고사운드,화질등 나무랄게 없어요
15                    밥 먹을때 자기전에 이동하면서 보고있는데 너무 편하고 좋아요
16                             lg기사님들도 친절하시고 tv도 좋아요!!!
17                                 상품에 만족하여 잘 사용

In [43]:
# 응답들을 저장할 리스트
content_list = []

total_iterations = len(test)

with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(test):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        input_raw_text = text
        system_message = {"role": "system", "content": sentiment_instructions}
        user_message = {"role": "user", "content": f"{sentiment_few_shots}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
        messages = [system_message, user_message]

        # GPT-4로부터 응답 받기
        content = get_response(messages)
        if not content:
            pbar.update(1)
            time.sleep(1)
            continue
        
        # 응답을 리스트에 저장
        content_list.append(content)
        
        pbar.update(1)
        time.sleep(1)

# content_list를 DataFrame으로 변환


100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


TypeError: the JSON object must be str, bytes or bytearray, not list

In [46]:
content_list

['{\n  "raw_text": "조아용 조아용 조아용 ㅎㅎ",\n  "issue": [\n    "만족"\n  ],\n  "sentiment": {\n    "만족": "긍정"\n  },\n  "sentiment_all": "긍정"\n}',
 '{\n  "raw_text": "너무 만족하며 사용중입니다",\n  "issue": [\n    "사용 만족"\n  ],\n  "sentiment": {\n    "사용 만족": "긍정"\n  },\n  "sentiment_all": "긍정"\n}',
 '{\n  "raw_text": "4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요",\n  "issue": [\n    "빠른 설치 및 배송",\n    "설치 서비스 만족"\n  ],\n  "sentiment": {\n    "빠른 설치 및 배송": "긍정",\n    "설치 서비스 만족": "긍정"\n  },\n  "sentiment_all": "긍정"\n}',
 '{\n  "raw_text": "여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요",\n  "issue": [\n    "여자친구 만족",\n    "선물로 추천"\n  ],\n  "sentiment": {\n    "여자친구 만족": "긍정",\n    "선물로 추천": "긍정"\n  },\n  "sentiment_all": "긍정"\n}',
 '{\n  "raw_text": "살까말까 했는데 좋아요",\n  "issue": [\n    "좋은 제품"\n  ],\n  "sentiment": {\n    "좋은 제품": "긍정"\n  },\n  "sentiment_all": "긍정"\n}',
 '{\n  "raw_text": "빠른 배송 감사합니다. LG에서 직잡 배송/설치 했습니다. .",\n  "issue": [\n    "빠른 배송",\n    "직접 배송/설치"\n  ],\n  "sentiment": {\n    "빠른 배송": "긍정",\n    "직접 배송

In [59]:
data_list = []

for content in content_list:
    parsed_content = convert_to_dict(content)

    raw_text = extract_raw_text(parsed_content)
    issues = extract_issues(parsed_content)
    sentiments = extract_sentiments(parsed_content)
    sentiment_all = extract_sentiment_all(parsed_content)

    for issue in issues:
        sentiment = sentiments.get(issue, 'Unknown')
        data_entry = {
            "raw_text": raw_text,
            "issue": issue,
            "sentiment": sentiment,
            "sentiment_all": sentiment_all
        }
        data_list.append(data_entry)

result = pd.DataFrame(data_list)
result

,raw_text,issue,sentiment,sentiment_all
0,조아용 조아용 조아용 ㅎㅎ,만족,긍정,긍정
1,너무 만족하며 사용중입니다,사용 만족,긍정,긍정
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 및 배송,긍정,긍정
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정
5,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,선물로 추천,긍정,긍정
6,살까말까 했는데 좋아요,좋은 제품,긍정,긍정
7,빠른 배송 감사합니다. LG에서 직잡 배송/설치 했습니다. .,빠른 배송,긍정,긍정
8,빠른 배송 감사합니다. LG에서 직잡 배송/설치 했습니다. .,직접 배송/설치,긍정,긍정
9,"자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확...",더 큰 화면으로 OTT 시청 좋음,긍정,긍정


# 적용 영역

In [60]:
df

,내용,구매처,평점,작성날짜,사용자,상품
0,조아용 조아용 조아용 ㅎㅎ,오아시스486,5,24.05.15.,rorn****,스탠바이미
1,너무 만족하며 사용중입니다,Ader,5,24.05.15.,arde****,스탠바이미
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,Ader,5,24.05.15.,jeon****,스탠바이미
3,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,Ader,5,24.05.15.,pjw9****,스탠바이미
4,살까말까 했는데 좋아요,11번가,4,24.05.15.,m0******,스탠바이미
...,...,...,...,...,...,...
995,선물용으로 구매했는데 빠른 배송 감사합니다.,LG전자,5,23.09.06.,c*0*****,스탠바이미고
996,사무실 휴대용으로 구매했습니다. 만족한것 같습니다,LG전자인증점 신영플러스,5,23.09.06.,fore****,스탠바이미고
997,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,알렛츠ALLETS,5,23.09.05.,gilg****,스탠바이미고
998,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,오늘의집,5,23.09.05.,1*6*****,스탠바이미고


### Step 1) gpt api 호출

In [61]:
text_apply = df['내용']

In [62]:
# 응답들을 저장할 리스트
content_list = []

total_iterations = len(text_apply)

with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(text_apply):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        input_raw_text = text
        system_message = {"role": "system", "content": sentiment_instructions}
        user_message = {"role": "user", "content": f"{sentiment_few_shots}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
        messages = [system_message, user_message]

        # GPT-4로부터 응답 받기
        content = get_response(messages)
        if not content:
            pbar.update(1)
            time.sleep(1)
            continue
        
        # 응답을 리스트에 저장
        content_list.append(content)
        
        pbar.update(1)
        time.sleep(1)

100%|██████████| 1000/1000 [44:43<00:00,  2.68s/it]


### Step 2) 답변 dataframe 변환

In [63]:
# 답변 변환

data_list = []

for content in content_list:
    parsed_content = convert_to_dict(content)

    raw_text = extract_raw_text(parsed_content)
    issues = extract_issues(parsed_content)
    sentiments = extract_sentiments(parsed_content)
    sentiment_all = extract_sentiment_all(parsed_content)

    for issue in issues:
        sentiment = sentiments.get(issue, 'Unknown')
        data_entry = {
            "raw_text": raw_text,
            "issue": issue,
            "sentiment": sentiment,
            "sentiment_all": sentiment_all
        }
        data_list.append(data_entry)

result = pd.DataFrame(data_list)
result

,raw_text,issue,sentiment,sentiment_all
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정
1,너무 만족하며 사용중입니다,만족,긍정,긍정
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정
...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정


### Step3) theme 생성(군집분석)

In [80]:
# issue에서 중복 제거
unique_issues = result['issue'].drop_duplicates().tolist()

In [94]:
len(unique_issues)

1153

In [98]:
def classify_issues(issues):
    # 'instructions'는 모델의 행동 방침을 설정하므로 system 메시지에 포함
    messages = [
        {"role": "system", "content": theme_instructions},
        {"role": "user", "content": f"다음 이슈들을 의미적으로 비슷한 그룹으로 나누고, 각 그룹의 공통 테마를 도출해줘: {issues}"},
        {"role": "user", "content": f"예시: {theme_few_shots}"}
    ]
    
    return get_response(messages)

### batch50

In [99]:
# 유니크한 이슈들을 GPT API로 분류
themes = []
batch_size = 50  # 한번에 너무 많은 데이터를 보내지 않기 위해 배치 처리
for i in range(0, len(unique_issues), batch_size):
    batch_issues = unique_issues[i:i+batch_size]
    themes.append(classify_issues(', '.join(batch_issues)))

# 배치 결과를 합쳐서 하나의 테마 리스트로 변환
theme_list = [theme.strip() for theme in ', '.join(themes).split(',')]

In [106]:
themes

['{\n  "제품 품질 및 성능": ["좋은 제품", "우수한 사운드", "우수한 화질", "좋은 TV", "품질 만족"],\n  "설치 및 배송 서비스": ["빠른 설치 배송", "설치 서비스 만족", "직접 배송 및 설치", "친절한 설치", "신속한 설치", "배송 설치", "친절한 배송", "친절한 설치 기사", "친절한 기사", "빠른 설치 안내", "조립 서비스"],\n  "가격 및 혜택": ["저렴한 가격", "좋은 가격", "할인"],\n  "사용자 만족도": ["만족", "여자친구 만족", "대만족", "제품 만족", "만족스러운 소비", "추천", "적극 추천", "부모님 선물 추천", "TV 미시청자 추천"],\n  "디자인 및 외관": ["디자인 만족", "예쁜 디자인"],\n  "편리성 및 사용성": ["이동 편리", "무겁지만 이동 용이", "편리한 터치", "터치 작동", "누워서 사용하기 좋음"],\n  "콘텐츠 시청 경험": ["더 큰 화면으로 OTT 시청 좋음", "OTT 시청", "OTT/유튜브 시청 좋음", "식사시간 시청", "자기전 시청"],\n  "서비스 및 고객 응대": ["좋은 서비스", "친절한 서비스"],\n  "기타": ["선물 아쉬움", "충전 번거로움", "리모컨", "빠른 구매", "설명 만족"]\n}',
 '{\n  "디자인 및 인테리어": ["훌륭한 디자인", "깔끔한 디자인", "인테리어와 조화", "뒷면 색상"],\n  "화질 및 음향": ["만족스러운 화질", "좋은 화질", "깨끗한 화면", "만족스러운 음량", "좋은 소리"],\n  "설치 및 이동": ["편리한 이동", "이동성", "조용한 이동", "섬세한 설치", "설치 만족", "이동식 티비 만족"],\n  "사용 편의성": ["실용성", "편리한 사용", "다양한 기기 사용", "OTT 시청 편리", "넷플릭스 시청", "TV없는 가정에서 유용", "지상파 미시청", "유료 어플 필요", "TV 시청 가능", "전용티비", "아이들

In [115]:
len(themes)

24

In [122]:
# 각 theme의 key를 추출하여 총 길이를 계산
total_keys_length = sum(len(json.loads(theme).keys()) for theme in themes)
total_keys_length

206

In [119]:
from collections import defaultdict

# themes 리스트의 각 JSON 문자열을 dict 형태로 변환하여 병합하는 함수
def merge_themes(themes):
    merged_dict = defaultdict(list)  # 리스트를 기본값으로 갖는 defaultdict 생성

    for theme in themes:
        # JSON 문자열을 Python dict로 변환
        theme_dict = json.loads(theme)
        
        # 각 key와 value를 병합
        for key, values in theme_dict.items():
            for value in values:
                if value not in merged_dict[key]:  # 중복된 value가 없을 경우만 추가
                    merged_dict[key].append(value)
    
    return dict(merged_dict)  # defaultdict을 일반 dict로 변환하여 반환

# 예시: themes 리스트에서 병합된 결과를 출력
merged_themes = merge_themes(themes)

In [123]:
len(merged_themes.keys()) # batch 결과 206개 -> merge 결과 169개

169

In [124]:
result

,raw_text,issue,sentiment,sentiment_all
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정
1,너무 만족하며 사용중입니다,만족,긍정,긍정
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정
...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정


In [126]:
result_test = result.copy()

In [127]:
# 'theme' 컬럼을 생성하는 함수
def find_theme(issue):
    for theme, issues in merged_themes.items():
        if issue in issues:
            return theme
    return '기타'  # 해당하는 theme이 없을 경우 '기타'로 표시

# 'theme' 컬럼을 생성
result_test['theme'] = result_test['issue'].apply(find_theme)
result_test

,raw_text,issue,sentiment,sentiment_all,theme
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,너무 만족하며 사용중입니다,만족,긍정,긍정,사용자 만족도
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,설치 및 배송 서비스
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,설치 및 배송 서비스
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,사용자 만족도
...,...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정,사용 환경
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,만족도
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,사용 환경
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,편리성 및 사용성


In [132]:
result_test.to_pickle('result/제공_네이버쇼핑리뷰_batch50_감성테마.pkl')

In [128]:
result_test.to_excel('result/제공_네이버쇼핑리뷰_감성테마_batch50_test1.xlsx', index=False)

### batch100

In [ ]:
# 유니크한 이슈들을 GPT API로 분류
themes = []
batch_size = 100  # 한번에 너무 많은 데이터를 보내지 않기 위해 배치 처리
for i in range(0, len(unique_issues), batch_size):
    batch_issues = unique_issues[i:i+batch_size]
    themes.append(classify_issues(', '.join(batch_issues)))

# 배치 결과를 합쳐서 하나의 테마 리스트로 변환
theme_list = [theme.strip() for theme in ', '.join(themes).split(',')]